<a href="https://colab.research.google.com/github/TebaSUH/Counrties-Classification/blob/main/GeoGuessr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading and spliting the data


In [ ]:
rom datasets import load_dataset, DatasetDict
import gc

print("Loading GeoGuessr dataset (streaming mode)...")
dataset = load_dataset("marcelomoreno26/geoguessr", split='train', streaming=True)

# Define countries
countries = ['Japan', 'France', 'Mexico'] #try Brazil instead of Mexico
samples_per_country = 400  #Mexico = 400

# Filter data
filtered_data = []
country_counts = {c: 0 for c in countries}

for item in dataset:
    country = item['label']
    if country in countries and country_counts[country] < samples_per_country:
        filtered_data.append({
            'image': item['image'],
            'label': countries.index(country)
        })
        country_counts[country] += 1

        # if sum(country_counts.values()) % 100 == 0:
        #     print(f"Progress: {country_counts}")

    if all(c >= samples_per_country for c in country_counts.values()):
        break

print(f"\nFinal counts: {country_counts}")
print(f"Total: {len(filtered_data)} images")

# Memory cleanup
del dataset
gc.collect()
print("Dataset loading complete, memory cleaned")

split to: train, validation and test

In [ ]:
from sklearn.model_selection import train_test_split

labels = [item['label'] for item in filtered_data]

# Split: 70% train, 15% validation, 15% test
train_val, test = train_test_split(filtered_data, test_size=0.15, stratify=labels, random_state=42)

train_val_labels = [item['label'] for item in train_val]
train, validation = train_test_split(train_val, test_size=0.176, stratify=train_val_labels, random_state=42)

print(f"Train: {len(train)}")
print(f"Validation: {len(validation)}")
print(f"Test: {len(test)}")

Plotting and Visualizing

In [ ]:
import matplotlib.pyplot as plt

# Show 5 example images
num_examples = 5
examples = train[:num_examples]

plt.figure(figsize=(15, 10))
for i, example in enumerate(examples):
    plt.subplot(1, num_examples, i + 1)
    image = example['image']
    label_idx = example['label']
    country = countries[label_idx]

    plt.imshow(image)
    plt.title(country, fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

prepare the model

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

checkpoint = "google/vit-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(checkpoint)
# Load base model
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(countries),
    ignore_mismatched_sizes=True
)